In [2]:
import cupy as cp

def compute_distance_transform(binary_tile):
    """
    Compute the distance transform for a binary tile on GPU using Chamfer distance.
    """
    # Ensure binary_tile is binary (0s and 1s)
    binary_tile = cp.asarray(binary_tile, dtype=cp.uint8)
    
    # Get dimensions
    z, y, x = binary_tile.shape
    
    # Initialize the distance transform array
    distance_transform = cp.full((z, y, x), cp.inf, dtype=cp.float32)

    # Create a temporary array to store the Chamfer distance
    chamfer_weights = cp.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]], dtype=cp.float32)

    # Forward pass: Check from top-left to bottom-right
    for i in range(z):
        for j in range(y):
            for k in range(x):
                if binary_tile[i, j, k] > 0:  # If it is an object pixel
                    distance_transform[i, j, k] = 0
                else:
                    # Check neighboring pixels
                    min_distance = distance_transform[i, j, k]
                    for dz in range(-1, 2):
                        for dy in range(-1, 2):
                            for dx in range(-1, 2):
                                if (0 <= i + dz < z) and (0 <= j + dy < y) and (0 <= k + dx < x):
                                    # Update the distance based on neighbors
                                    min_distance = min(min_distance, distance_transform[i + dz, j + dy, k + dx] + chamfer_weights[dz + 1, dy + 1])
                    distance_transform[i, j, k] = min_distance

    # Backward pass: Check from bottom-right to top-left
    for i in range(z - 1, -1, -1):
        for j in range(y - 1, -1, -1):
            for k in range(x - 1, -1, -1):
                if binary_tile[i, j, k] == 0:  # Only update distance for background pixels
                    min_distance = distance_transform[i, j, k]
                    for dz in range(-1, 2):
                        for dy in range(-1, 2):
                            for dx in range(-1, 2):
                                if (0 <= i + dz < z) and (0 <= j + dy < y) and (0 <= k + dx < x):
                                    # Update the distance based on neighbors
                                    min_distance = min(min_distance, distance_transform[i + dz, j + dy, k + dx] + chamfer_weights[dz + 1, dy + 1])
                    distance_transform[i, j, k] = min_distance

    return distance_transform


In [ ]:
import numpy as np
import cupy as cp
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys

# Load binary volume
binary_volume = tiff.imread("outputs/output_volume.tif")
print(f'Size of binary volume: {sys.getsizeof(binary_volume)} bytes')

# Skeletonize directly on the CPU volume
skeleton_volume = skeletonize(binary_volume)  # Skeletonize the binary volume
print(f'Skeletonization complete. Size of skeleton volume: {sys.getsizeof(skeleton_volume)} bytes')

# Transfer binary and skeleton volumes to GPU
binary_volume_gpu = cp.asarray(binary_volume, dtype=cp.uint8)
skeleton_volume_gpu = cp.asarray(skeleton_volume, dtype=cp.uint8)
print("Volumes transferred to GPU")

# Define tile size to process portions of the volume at a time
tile_size = (64, 64, 64)  # Adjust based on available memory and volume size

def calculate_local_thickness_tile(binary_tile, skeleton_tile, max_distance=10):
    """
    Calculates the local thickness for a single tile.
    """
    # Convert binary tile to float
    binary_tile_gpu = cp.asarray(binary_tile, dtype=cp.float32)
    
    # Compute distance transform (using an appropriate method)
    # For example, pseudo-code, since CuPy does not have a built-in function
    distance_transform_tile_gpu = compute_distance_transform(binary_tile_gpu)  # Placeholder function
    
    # Apply skeleton mask on GPU
    thickness_tile_gpu = cp.where(skeleton_tile > 0, distance_transform_tile_gpu, cp.inf)
    
    # Mask values greater than max_distance
    thickness_tile_gpu[thickness_tile_gpu > max_distance] = 0
    
    return thickness_tile_gpu

# Process the volume in tiles
thickness_map = np.zeros_like(binary_volume, dtype=np.float32)  # Output thickness map on CPU

# Calculate the dimensions of the tiles
z_tiles = (binary_volume.shape[0] + tile_size[0] - 1) // tile_size[0]
y_tiles = (binary_volume.shape[1] + tile_size[1] - 1) // tile_size[1]
x_tiles = (binary_volume.shape[2] + tile_size[2] - 1) // tile_size[2]

# Total number of tiles
total_tiles = z_tiles * y_tiles * x_tiles
current_tile = 0

# Calculate dimensions for tiling
for z in range(0, binary_volume.shape[0], tile_size[0]):
    for y in range(0, binary_volume.shape[1], tile_size[1]):
        for x in range(0, binary_volume.shape[2], tile_size[2]):
            # Define tile limits
            z_end = min(z + tile_size[0], binary_volume.shape[0])
            y_end = min(y + tile_size[1], binary_volume.shape[1])
            x_end = min(x + tile_size[2], binary_volume.shape[2])

            # Extract tile from the binary and skeleton volumes
            binary_tile_gpu = binary_volume_gpu[z:z_end, y:y_end, x:x_end]
            skeleton_tile_gpu = skeleton_volume_gpu[z:z_end, y:y_end, x:x_end]

            # Calculate local thickness for this tile
            thickness_tile_gpu = calculate_local_thickness_tile(binary_tile_gpu, skeleton_tile_gpu)
            
            # Transfer result back to CPU and store in the final thickness map
            thickness_map[z:z_end, y:y_end, x:x_end] = cp.asnumpy(thickness_tile_gpu)
            
            # Clear GPU memory for the next tile
            del binary_tile_gpu, skeleton_tile_gpu, thickness_tile_gpu
            cp._default_memory_pool.free_all_blocks()

            # Update and print progress
            current_tile += 1
            progress = (current_tile / total_tiles) * 100
            if current_tile % 1000 == 0 or current_tile == total_tiles:  # Print every 100 tiles or last tile
                print(f"Progress: {progress:.2f}%")

print("Thickness map calculated and combined on CPU.")

# Instantiate skeleton object from skeleton volume
skeleton = Skeleton(skeleton_volume)  # Define skeleton from the CPU volume



Size of binary volume: 2666222244 bytes
Skeletonization complete. Size of skeleton volume: 144 bytes
Volumes transferred to GPU
